# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
T. Henning  ->  T. Henning  |  ['T. Henning']
M. Benisty  ->  M. Benisty  |  ['M. Benisty']
W. Brandner  ->  W. Brandner  |  ['W. Brandner']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
P. Garcia  ->  A. P. Garcia  |  ['P. Garcia']
L. Kreidberg  ->  L. Kreidberg  |  ['L. Kreidberg']
P. Mollière  ->  P. Mollière  |  ['P. Mollière']
M. Samland  ->  M. Samland  |  ['M. Samland']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
J. Shi  ->  J. Shi  |  ['J. Shi']
Arxiv has 67 new papers today
          5 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/5 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2510.08675


extracting tarball to tmp_2510.08675... done.
Retrieving document from  https://arxiv.org/e-print/2510.08691


extracting tarball to tmp_2510.08691... done.


Found 140 bibliographic references in tmp_2510.08691/aa56860-25.bbl.
Issues with the citations
syntax error in line 5: unbalanced braces
Retrieving document from  https://arxiv.org/e-print/2510.09046


...

✔ → 140504:\section{Validation of spectroscopic galaxy clustering 2-point correlation function}\label{app:a}
  ↳ 141569:\section{The shear power spectrum in the Limber approximation}


✔ → 141569:\section{The shear power spectrum in the Limber approximation}
  ↳ 146471:\section{Approximating the cb--total matter correlator}
✔ → 146471:\section{Approximating the cb--total matter correlator}
  ↳ 148987:end


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/rel_diff_xi_g_mu.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{Figures/rel_diff_xi_g_mu.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure Figures/rel_diff_xi_mu_mu.pdf
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\columnwidth]{Figures/rel_diff_xi_mu_mu.pdf}
  warnings.warn(LatexWarning(f"Could not find graphic {

Found 106 bibliographic references in tmp_2510.09147/main.bbl.
Issues with the citations
syntax error in line 326: '=' expected
Retrieving document from  https://arxiv.org/e-print/2510.09600
extracting tarball to tmp_2510.09600... done.


/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.23/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2510.09600/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.08691-b31b1b.svg)](https://arxiv.org/abs/2510.08691) | **The ExoGRAVITY survey: a K-band spectral library of giant exoplanet and brown dwarf companions**  |
|| J. Kammerer, et al. -- incl., <mark>T. Henning</mark>, <mark>M. Benisty</mark>, <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark>, <mark>M. Samland</mark> |
|*Appeared on*| *2025-10-13*|
|*Comments*| *21 pages, 11 figures, accepted for publication in A&A*|
|**Abstract**|            Direct observations of exoplanet and brown dwarf companions with near-infrared interferometry, first enabled by the dual-field mode of VLTI/GRAVITY, provide unique measurements of the objects' orbital motion and atmospheric composition. Here, we compile a homogeneous library of all exoplanet and brown dwarf K-band spectra observed by GRAVITY thus far. We re-reduce all available GRAVITY dual-field high-contrast data and, where companions are detected, extract their ~2.0-2.4 $\mu$m K-band contrast spectra. We then derive stellar model atmospheres for all employed flux references that we use to convert the companion contrast into companion flux spectra. Solely from the resulting GRAVITY spectra, we extract spectral types, spectral indices, and bulk physical properties for all companions. Finally, and with the help of age constraints from the literature, we also derive isochronal masses for most companions using evolutionary models. The resulting library contains R ~ 500 GRAVITY spectra of 39 substellar companions from late M to late T spectral type, including the entire L-T transition. Throughout this transition, a shift from CO-dominated late M- and L-type dwarfs to CH4-dominated T-type dwarfs can be observed in the K-band. The GRAVITY spectra alone constrain the objects' bolometric luminosity to typically within $\pm$0.15 dex. The derived isochronal masses agree with dynamical masses from the literature where available, except for HD 4113 c for which we confirm its previously reported potential underluminosity. Medium resolution spectroscopy of substellar companions with GRAVITY provides insight into the carbon chemistry and the cloudiness of these objects' atmospheres. It also constrains these objects' bolometric luminosity which can yield measurements of their formation entropy if combined with dynamical masses, for instance from Gaia and GRAVITY astrometry.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.09147-b31b1b.svg)](https://arxiv.org/abs/2510.09147) | **Euclid preparation. Cosmology Likelihood for Observables in Euclid (CLOE). 5. Extensions beyond the standard modelling of theoretical probes and systematic effects**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2025-10-13*|
|*Comments*| **|
|**Abstract**|            Euclid is expected to establish new state-of-the-art constraints on extensions beyond the standard LCDM cosmological model by measuring the positions and shapes of billions of galaxies. Specifically, its goal is to shed light on the nature of dark matter and dark energy. Achieving this requires developing and validating advanced statistical tools and theoretical prediction software capable of testing extensions of the LCDM model. In this work, we describe how the Euclid likelihood pipeline, Cosmology Likelihood for Observables in Euclid (CLOE), has been extended to accommodate alternative cosmological models and to refine the theoretical modelling of Euclid primary probes. In particular, we detail modifications made to CLOE to incorporate the magnification bias term into the spectroscopic two-point correlation function of galaxy clustering. Additionally, we explain the adaptations made to CLOE's implementation of Euclid primary photometric probes to account for massive neutrinos and modified gravity extensions. Finally, we present the validation of these CLOE modifications through dedicated forecasts on synthetic Euclid-like data by sampling the full posterior distribution and comparing with the results of previous literature. In conclusion, we have identified in this work several functionalities with regards to beyond-LCDM modelling that could be further improved within CLOE, and outline potential research directions to enhance pipeline efficiency and flexibility through novel inference and machine learning techniques.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.08675-b31b1b.svg)](https://arxiv.org/abs/2510.08675) | **How precisely can we measure the ages of subgiant and giant stars?**  |
|| C. Shariat, <mark>K. El-Badry</mark>, S. Bhattacharjee |
|*Appeared on*| *2025-10-13*|
|*Comments*| *5 pages, 1 figure. Submitted to PASP, comments are welcome*|
|**Abstract**|            Precise stellar ages are fundamental to Galactic archaeology. However, obtaining reliable age estimates and uncertainties for field stars has been a long-standing challenge. We test the fidelity of ages from recent catalogs of giants and subgiants using wide binaries, whose components formed at the same time and thus should have consistent inferred ages. We find that subgiant ages based on spectroscopic metallicities from Xiang & Rix (2022) are generally consistent within their reported uncertainties, implying that fractional uncertainties of 5-10% are realistically achievable. In contrast, we find that published photometric subgiant ages and red giant ages underestimate true uncertainties by factors of 2-5. These results demonstrate that accurate metallicity and $\alpha$-element abundances are essential for precise isochrone ages and establish wide binaries as a powerful, model-independent benchmark for calibrating stellar age measurements in the era of large spectroscopic surveys.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.09046-b31b1b.svg)](https://arxiv.org/abs/2510.09046) | **A study of 80 known pulsars at 185 MHz using MWA incoherent drift-scan observations**  |
|| T. Yu, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2025-10-13*|
|*Comments*| *16 pages, 9 figures, accepted for publication in MNRAS*|
|**Abstract**|            A systematic study of 80 known pulsars observed at 185 MHz has been conducted using archival incoherent-sum data from the Murchison Widefield Array (MWA). The dataset comprises 48 drift-scan observations from the MWA Voltage Capture System, covering approximately 30,000 square degrees of sky with sensitivities reaching about 8 mJy in the deepest regions. An optimized PRESTO-based search pipeline was deployed on the China SKA Regional Centre infrastructure. This enabled the detection of 80 known pulsars, representing a 60 percent increase over the previous census. Notably, this includes 30 pulsars with first-time detections at this frequency, of which pulse profiles and flux densities are presented. Spectral, scattering, and pulse-width properties were examined for the sample, providing observational constraints on low-frequency turnover, propagation effects, and width-period relations. This study highlights the value of wide-field, low-frequency time-domain surveys for constraining pulsar emission and propagation, offering empirical insights that may inform future observations with instruments such as SKA-Low.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2510.09600-b31b1b.svg)](https://arxiv.org/abs/2510.09600) | **JWST Observations of SN 2024ggi II: NIRSpec Spectroscopy and CO Modeling at 285 and 385 Days Past the Explosion**  |
|| T. Mera, et al. -- incl., <mark>J. Shi</mark> |
|*Appeared on*| *2025-10-13*|
|*Comments*| *ApJ (submitted)*|
|**Abstract**|            We present James Webb Space Telescope (JWST) NIRSpec 1.7--5.5 micron observations of SN~2024ggi at +285.51 and +385.27 days post-explosion. The late-time nebular spectra are dominated by emission lines from various ionization states of H, Ca, Ar, C, Mg, Ni, Co, and Fe. We also detect strong CO emission in both the first overtone and fundamental vibrational bands. Most atomic features exhibit asymmetric line profiles, indicating an aspherical explosion. Using observed fluxes combined with non-LTE radiative-transfer simulations, we develop a data-driven method that resolves the complex molecular-emission region, constrains its 3D structure, and reproduces high-fidelity spectral profiles. We find that, CO is mostly formed prior to +285d past explosion. The subsequent evolution is dominated by the evaporation of CO with CO mass varying from M(CO) of 8.7E-3 to 1.3E-3 Mo, and with instabilities growing from almost homogeneous to highly clumped (density contrast f_c of 1.2 to 2). The minimum velocity of CO only slightly decreases between epochs (v_1 of 1200 and 1100 km/sec), with the reference temperature dropping from T_1 of 2400 and 1900K.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2510.08691/./figures/HD206505_b_all.png', 'tmp_2510.08691/./figures/gravity_233_zoom.png', 'tmp_2510.08691/./figures/all_combined_spectra_good_plain.png', 'tmp_2510.08691/./figures/evolutionary_models.png', 'tmp_2510.08691/./figures/mass_correlation.png']
copying  tmp_2510.08691/./figures/HD206505_b_all.png to _build/html/
copying  tmp_2510.08691/./figures/gravity_233_zoom.png to _build/html/
copying  tmp_2510.08691/./figures/all_combined_spectra_good_plain.png to _build/html/
copying  tmp_2510.08691/./figures/evolutionary_models.png to _build/html/
copying  tmp_2510.08691/./figures/mass_correlation.png to _build/html/
exported in  _build/html/2510.08691.md
    + _build/html/tmp_2510.08691/./figures/HD206505_b_all.png
    + _build/html/tmp_2510.08691/./figures/gravity_233_zoom.png
    + _build/html/tmp_2510.08691/./figures/all_combined_spectra_good_plain.png
    + _build/html/tmp_2510.08691/./figures/evolutionary_models.png
    + _build/html/tmp_2510.08691/./figures

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\micron}{\text{\textmu m}}$
$\newcommand{\red}[1]{\textcolor{black}{#1}}$</div>



<div id="title">

# The ExoGRAVITY survey: a _K_-band spectral library of giant exoplanet and brown dwarf companions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.08691-b31b1b.svg)](https://arxiv.org/abs/2510.08691)<mark>Appeared on: 2025-10-13</mark> -  _21 pages, 11 figures, accepted for publication in A&A_

</div>
<div id="authors">

J. Kammerer, et al. -- incl., <mark>T. Henning</mark>, <mark>M. Benisty</mark>, <mark>W. Brandner</mark>, <mark>G. Chauvin</mark>, <mark>P. Garcia</mark>, <mark>L. Kreidberg</mark>, <mark>P. Mollière</mark>, <mark>M. Samland</mark>

</div>
<div id="abstract">

**Abstract:** Direct observations of exoplanet and brown dwarf companions with near-infrared interferometry, first enabled by the dual-field mode of VLTI/GRAVITY, provide unique measurements of the objects' orbital motion and atmospheric composition. Here, we compile a homogeneous library of all exoplanet and brown dwarf _K_ -band spectra observed by GRAVITY thus far. This ExoGRAVITY Spectral Library is made publicly available online. $\red{We re-reduce all available GRAVITY dual-field high-contrast data in a uniform and highly automated way and, where companions are detected, extract their $\sim2.0$--$2.4 \micron$ _K_-band contrast spectra.}$ We then derive stellar model atmospheres for all employed flux references (either the host star or the swap calibrator) that we use to convert the companion contrast into companion flux spectra. Solely from the resulting GRAVITY _K_ -band flux spectra, we extract spectral types, spectral indices, and bulk physical properties for all companions. Finally, and with the help of age constraints from the literature, we also derive isochronal masses for most companions using evolutionary models. The resulting library contains $R \sim 500$ GRAVITY _K_ -band spectra of 39 substellar companions from late M to late T spectral type, including the entire L-T transition. Throughout this transition, a shift from CO-dominated late M- and L-type dwarfs to CH $ _4$ -dominated T-type dwarfs can be observed in the _K_ -band. The GRAVITY spectra alone constrain the objects' bolometric luminosity to typically within $\pm0.15$ dex. The derived isochronal masses agree with dynamical masses from the literature where available, except for HD 4113 c for which we confirm its previously reported potential underluminosity. Medium resolution spectroscopy of substellar companions with GRAVITY provides insight into the carbon chemistry and the cloudiness of these objects' atmospheres. It also constrains these objects' bolometric luminosity which can yield measurements of their formation entropy if combined with dynamical masses, for instance from Gaia and GRAVITY astrometry.

</div>

<div id="div_fig1">

<img src="tmp_2510.08691/./figures/HD206505_b_all.png" alt="Fig1.1" width="50%"/><img src="tmp_2510.08691/./figures/gravity_233_zoom.png" alt="Fig1.2" width="50%"/>

**Figure 1. -** $\red${Combination of the GRAVITY epochs of HD 206505 b after scaling them to bring them into mutual agreement.}_Top panel_: GRAVITY _K_-band flux spectra for each individual epoch. The $3\sigma$ flux uncertainties are shown as shaded regions. The legend reveals that for this object, both observations were done in dual-field on-axis mode, with the companion located at an angular separation of $425$ mas from the host star. _Bottom panel_: Scaled GRAVITY flux spectra for each individual epoch and the resulting combined spectrum (black line). The legend reveals the scale factors applied to each epoch (values in brackets). (*fig:hd206505_b_all*)

</div>
<div id="div_fig2">

<img src="tmp_2510.08691/./figures/all_combined_spectra_good_plain.png" alt="Fig6" width="100%"/>

**Figure 6. -** GRAVITY _K_-band spectra at medium spectral resolution ($R \sim 500$) of the 22 substellar companions with well-constrained spectral type, sorted from M-type (top) to T-type (bottom). The spectra are scaled with respect to each other by the ratio of the square root of their SNR and their mean flux, and they are offset along the y-axis so that their mean value is located at the height of their corresponding tick mark for better readability. The best fitting spectral type templates from the SpeX Prism Libraries are shown in gray with their corresponding _K1_-_K2_ extinction $E_K$ in units of magnitudes shown on the left. The SNR of each spectrum is shown on the right. (*fig:all_combined_spectra_good*)

</div>
<div id="div_fig3">

<img src="tmp_2510.08691/./figures/evolutionary_models.png" alt="Fig8.1" width="50%"/><img src="tmp_2510.08691/./figures/mass_correlation.png" alt="Fig8.2" width="50%"/>

**Figure 8. -** _Left panel_: Sonora Diamondback hybrid gravity solar metallicity (\citealp{morley2024}) and BHAC15 (\citealp{baraffe2015}) cooling curves for substellar companions. The 37 objects from the ExoGRAVITY Spectral Library with age constraints available in the literature are shown by red points, and those with dynamical mass measurements have a black contour. Cooling curves for a few masses are highlighted in blue (1 m$\text{M}_\text{sol} = 1.048 \text{M}_\text{Jup}$). _Right panel_: Correlation between dynamical mass constraints from the literature and evolutionary mass constraints from GRAVITY. HD 984 b lies outside the shown mass range but is still consistent between its evolutionary and dynamical mass to within $2\sigma$. (*fig:evolutionary_models*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.08691"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\kpnu}{EP\nu\xspace}$
$\newcommand{\dnnu}{\ensuremath{\Delta N_\mathrm{eff}}\xspace}$
$\newcommand{\pcb}{P_\mathrm{cb}}$
$\newcommand{\pmm}{P_\mathrm{m}}$
$\newcommand{\threetimestwo}{3\texttimes2pt\xspace}$
$\newcommand{\MyRed}{\color[rgb]{0.57, 0.0, 0.04}}$
$\newcommand{\arn}[1]{{\color[rgb]{1,0.5,0}#1}}$
$\newcommand{\arc}[1]{{\color[rgb]{1,0.5,0}{\sf[Ahmad: #1]}}}$
$\newcommand{\ars}[2]{{\color[rgb]{1,0.5,0}{\sout{#1}{#2}}}}$
$\newcommand{\sefa}[1]{\textcolor{lime}{#1}}$
$\newcommand{\sefout}[1]{\sefa{\sout{#1}}}$
$\newcommand{\sefaok}{\sefa{[\textbf{SP}:\surd]}}$
$\newcommand{\benb}[1]{\textcolor{darkb}{#1}}$
$\newcommand{\fsorr}[1]{{\color{magenta}{\sf[FS: #1]}}}$
$\newcommand{\fsort}[1]{{\color{magenta}{#1}}}$
$\newcommand{\fkeil}[1]{\textcolor{teal}{#1}}$
$\newcommand{\santi}[1]{{\color{orange}{\sf[Santi: #1]}}}$
$\newcommand{\cloe}{\texttt{CLOE}\xspace}$
$\newcommand{\orcid}[1]$
$\newcommand{\arraystretch}{1.5}$
$\newcommand\fraC{#1}$
$\newcommand\fra{#1}$
$\newcommand\fraRem{#1}$
$\newcommand\benbc{#1}$
$\newcommand\mmt{#1}$
$\newcommand\mmc{#1}$
$\newcommand\gfa{#1}$
$\newcommand\gch{#1}$
$\newcommand\ZSt{#1}$
$\newcommand\ZSc{#1}$
$\newcommand\etc{#1}$
$\newcommand\et{#1}$
$\newcommand\itt{#1}$
$\newcommand\itc{#1}$</div>



<div id="title">

# $\Euclid$ preparation: Cosmology Likelihood for Observables in Euclid (CLOE). 5. Extensions beyond the standard modelling of theoretical probes and systematic effects

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2510.09147-b31b1b.svg)](https://arxiv.org/abs/2510.09147)<mark>Appeared on: 2025-10-13</mark> - 

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** $\Euclid$ is expected to establish new state-of-the-art constraints on extensions beyond the standard $\LCDM$ cosmological model by measuring the positions and shapes of billions of galaxies. Specifically, its goal is to shed light on the nature of dark matter and dark energy. Achieving this requires developing and validating advanced statistical tools and theoretical prediction software capable of testing extensions of the $\LCDM$ model. In this work, we describe how the $\Euclid$ likelihood pipeline, Cosmology Likelihood for Observables in Euclid ( $\cloe$ ), has been extended to accommodate alternative cosmological models and to refine the theoretical modelling of $\Euclid$ primary probes. In particular, we detail modifications made to $\cloe$ to incorporate the magnification bias term into the spectroscopic two-point correlation function of galaxy clustering. Additionally, we explain the adaptations made to $\cloe$ 's implementation of $\Euclid$ primary photometric probes to account for massive neutrinos and modified gravity extensions. Finally, we present the validation of these $\cloe$ modifications through dedicated forecasts on synthetic $\Euclid$ -like data by sampling the full posterior distribution and comparing with the results of previous literature. In conclusion, we have identified in this work several functionalities with regards to beyond- $\LCDM$ modelling that could be further improved within $\cloe$ , and outline potential research directions to enhance pipeline efficiency and flexibility through novel inference and machine learning techniques.

</div>

<div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2510.09147"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

125  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

11  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
